## Creating Data for using this Notebook

The data is 10 M rows, 4 columns each file.

In [9]:
import cudf
import numpy as np

# Generate random data into cudf dataframe
def random_df(nrows):
    return cudf.DataFrame({
        'a': np.random.binomial(n=1, p=0.5, size=(nrows,)),
        'b': np.random.normal(size=(nrows,)),
        'c': np.random.normal(size=(nrows,)),
        'd': np.random.normal(size=(nrows,))
    })

random_df(10_000_000).to_csv('0.csv', index=False)
random_df(10_000_000).to_csv('1.csv', index=False)


## Imports and creating Blazing Context

In [10]:
from blazingsql import BlazingContext # import blazing context
bc = BlazingContext() # create blazing context


BlazingContext ready


## Creating Tables

In [11]:
## Creating tables from files.
## This we can create tables in Blazing directly of disk by pointing to locations where hive stores the data
## Can read directly from Prod, GCP...

## At this point data is not loaded into the GPU memory
## Blazing only create a pointer to read the data from the memory.
## Bleazing peeks into the data and creates a meta data for this table
bc.create_table('A', '/root/code/*.csv') 


In [12]:
## Running select * from A
q= '''
SELECT * FROM A
'''

res = bc.sql(q)
res

,a,b,c,d
0,1,0.728617,-0.846392,0.176448
1,0,-0.093679,-1.782987,-0.042896
2,1,0.194267,0.574008,0.308305
3,0,0.784187,1.017407,-2.183384
4,1,-0.020245,-1.046001,0.267910
...,...,...,...,...
19999995,1,0.035239,0.102899,-1.461216
19999996,1,0.283887,-0.484700,1.994155
19999997,0,-0.032769,0.189657,0.762399
19999998,1,1.543419,0.809247,0.176667


In [13]:
## Reunning Queries with CTEs

q= '''

WITH

cte1 as (
    SELECT * FROM A
    WHERE A.a = 1
)

SELECT * FROM cte1
'''

res = bc.sql(q)
res

,a,b,c,d
0,1,0.728617,-0.846392,0.176448
1,1,0.194267,0.574008,0.308305
2,1,-0.020245,-1.046001,0.267910
3,1,1.585304,-2.608947,0.716433
4,1,1.584202,1.665679,-0.866326
...,...,...,...,...
9997294,1,0.698527,-1.141175,2.048563
9997295,1,0.637495,-0.753099,-0.170067
9997296,1,0.035239,0.102899,-1.461216
9997297,1,0.283887,-0.484700,1.994155


In [14]:
## Casting a as boolean
q= '''
SELECT 
    CAST(a as boolean) as a, b, c, d
FROM 
    A
'''

res = bc.sql(q)
res

,a,b,c,d
0,True,0.728617,-0.846392,0.176448
1,False,-0.093679,-1.782987,-0.042896
2,True,0.194267,0.574008,0.308305
3,False,0.784187,1.017407,-2.183384
4,True,-0.020245,-1.046001,0.267910
...,...,...,...,...
19999995,True,0.035239,0.102899,-1.461216
19999996,True,0.283887,-0.484700,1.994155
19999997,False,-0.032769,0.189657,0.762399
19999998,True,1.543419,0.809247,0.176667


In [15]:
## Sorting on column 'a' , order by

q= '''
SELECT 
    * 
FROM A
    ORDER BY a
'''
bc.sql(q)

,a,b,c,d
0,0,-0.093679,-1.782987,-0.042896
1,0,0.784187,1.017407,-2.183384
2,0,0.016485,0.445144,0.413584
3,0,-0.156178,-1.050412,2.312639
4,0,-1.124699,0.781675,-0.894808
...,...,...,...,...
19999995,1,0.698527,-1.141175,2.048563
19999996,1,0.637495,-0.753099,-0.170067
19999997,1,0.035239,0.102899,-1.461216
19999998,1,0.283887,-0.484700,1.994155


In [16]:
## Using COALESCE functions

q= '''
SELECT a, COALESCE(a, b*2) 
FROM A
'''
bc.sql(q)

,a,EXPR$1
0,1,1.0
1,0,0.0
2,1,1.0
3,0,0.0
4,1,1.0
...,...,...
19999995,1,1.0
19999996,1,1.0
19999997,0,0.0
19999998,1,1.0


In [18]:
# group_by in column 'a', and aggrigate on other columns and rename the columns apropriately
q= '''
SELECT 
    a, max(b) as maximum, min(c) as minimum, sum(d) as TOTAL 
FROM A
    GROUP BY a
'''
res = bc.sql(q)
res

,a,maximum,minimum,TOTAL
0,1,5.521751,-4.986656,3035.960658
1,0,5.565807,-5.333017,17.054091


In [19]:
## in this cell take the result of the previous query, create a table of the query
## Join it to the initial data on columns 'a'

## Create a table from a cudf DataFrame
bc.create_table('B', res)

q= '''
SELECT * FROM A
JOIN B
    ON A.a = B.a
'''
bc.sql(q)

,a,b,c,d,a0,maximum,minimum,TOTAL
0,1,-1.091495,-0.754183,-0.933501,1,5.521751,-4.986656,3035.960658
1,0,-1.385782,0.744120,-0.855489,0,5.565807,-5.333017,17.054091
2,1,0.717433,-1.026488,0.702928,1,5.521751,-4.986656,3035.960658
3,1,2.041260,1.365403,1.538553,1,5.521751,-4.986656,3035.960658
4,1,0.148374,0.191762,0.319793,1,5.521751,-4.986656,3035.960658
...,...,...,...,...,...,...,...,...
19999995,0,-0.333474,-0.815204,-0.214769,0,5.565807,-5.333017,17.054091
19999996,1,1.424275,-0.288886,-0.692066,1,5.521751,-4.986656,3035.960658
19999997,1,0.823493,0.716913,0.841602,1,5.521751,-4.986656,3035.960658
19999998,1,-0.711562,-0.711718,0.329357,1,5.521751,-4.986656,3035.960658


In [20]:
## Gettting all the table that blazing context holds ponters too.
bc.tables

{'A': <pyblazing.apiv2.context.BlazingTable at 0x7f2bbc04e110>,
 'B': <pyblazing.apiv2.context.BlazingTable at 0x7f2bd641c850>}

## Exercise - 4 mins

Join Table A and Table B on column 'a', create 2 more new columns, named maxratio, minratio where
maxratio = maximum/TOTAL
minratio = minimum/TOTAL

and write the results to orc file "res.orc" ## hint use .to_csv(filename)


## Dropping Tables

In [21]:
## After running your queries, if you do not the the table in the memory anymore, please use the drop tables
## to clear the data in GPU memory, creatng more space in GPU for other processing.

bc.drop_table('A')
bc.drop_table('B')

## Creating Tables from HDFS (The following cell will not run as the node needs Kerb authentication)

In [ ]:
## Once your container is kerberized, you can add the hdfs to blazing context,
## by passing the following information to the below API 
bc.hdfs('Node', 
        host='<Name Node>', 
        port=8020, 
        user='<uname>',
        kerb_ticket="/tmp/krb5cc_0")

## Here we are creating a table only from the data we need, rather than poinitng to the whole table.
## Rather than reading all dates of data and filtering of the data we don't need, we are taking advantage of the 
## partitions hive makes and using it to read the data we only need. This works only when the data is pre-partitioned.

## Ex: Can read partitioned data, supposed we have 100 days of data partitioned by and we want to just read two days of data,
## We can provide partitions dictionary to read only the data we need.
bc.create_table('table1', fs, file_format='orc', 
                    partitions = {'visit_dt':['2020-04-05', '2020-04-06'], 'op_cmpny_cd':['WMT-US']},
                    partitions_schema = [('visit_dt','date'),('op_cmpny_cd','string')])